In [228]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

#from gensim.models import Word2Vec
#from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os


In [229]:
con = sqlite3.connect('database.sqlite') 

In [230]:
filtered_data = pd.read_sql_query("""select * from REVIEWS where SCORE!=3 LIMIT 5000""",con)

In [231]:
def partition(x):
    if x<3 : 
        return 0
    return 1

In [232]:
filtered_data.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [233]:
actualScore=filtered_data['Score']
positiveNegative=actualScore.map(partition)
positiveNegative

0       1
1       0
2       1
3       0
4       1
       ..
4995    0
4996    1
4997    1
4998    0
4999    0
Name: Score, Length: 5000, dtype: int64

In [234]:
filtered_data['Score']=positiveNegative
filtered_data.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


## Data Cleaning

In [235]:
pd.read_sql_query("""select * from REVIEWS group by ProfileName  """,con)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,136277,B000W4OQRW,ABCOLENOSM0IQ,,2,2,5,1232841600,great taste !!!,If I would have known this snack would be so g...
1,241773,B0081XPTBS,A2JKR0W0EQ6QQM,,0,2,5,1334966400,good formula!!,this formula can help baby happy every day<br ...
2,35173,B001ATMQK2,AD67UXV6RV1CV,,1,1,5,1317427200,SO GOOD!,This sauce is so good! Ordered it and it arriv...
3,172867,B000FAIRCI,A2NY8YRUTRWB0Y,:Jury Duty:,3,4,4,1268784000,Should not have to search all over the interne...,"After a 20 minute web search, here are the ing..."
4,5386,B000H27K86,A1ER6IYOMM8VCT,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",0,0,5,1199491200,"A TASTE OF ""N'AW'INS"" GREAT SPICE MIX FOR MANY...",I have been using this spice mix for many year...
...,...,...,...,...,...,...,...,...,...,...
218413,313629,B000VXYV6K,A3UZR5GVMUN0OH,§§§ Raven §§§,4,9,1,1294444800,My dogs ignore it.,Well I have dogs that have figured out that if...
218414,77461,B000ITUEBS,A3FLMS0NRJ8AWQ,Équité,0,2,1,1317427200,Too spicy.,I intended to eat the tuna with salad for lunc...
218415,32033,B0062A87HA,A16V6UJZPEYTPX,Étaín d'Orphée,1,1,5,1327795200,Great Pumpkin Feast,I ordered this recently when it when I noticed...
218416,56926,B000VK4CT4,A13SV7V95KQBXQ,"åmâz&#335;ni&#257;n ""Being an Earthling, Livi...",0,0,4,1334102400,"Very edible, too small though","If you have lived in Mainland China, you would..."


## Need to delete the dublicates 
 1. So, we first sort the data according to product id
 2. And then , drop all the dublicates

In [236]:
data_after_sorting=filtered_data.sort_values("ProductId",axis=0,ascending=True,inplace=False, kind='quicksort', na_position='last')

In [237]:
type(data_after_sorting)
unique_data=data_after_sorting.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first',inplace=False)
print("Size of Dublicated data : ",end='') 
print(data_after_sorting.shape)
print("Size of Deduplicated data : ",end='') 
print(unique_data.shape)

Size of Dublicated data : (5000, 10)
Size of Deduplicated data : (4986, 10)


In [238]:
## Percentage of Remaining Data

In [239]:
(unique_data.shape[0]/((filtered_data.shape[0]*1.0)))*100

99.72

In [240]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [241]:
print(unique_data.head(3)['Text'])

2546    Why is this $[...] when the same product is av...
2547    We have used the Victor fly bait for 3 seasons...
1145    I just received my shipment and could hardly w...
Name: Text, dtype: object


## Bag of Words 

In [242]:
count_vect=CountVectorizer();
final_counts = count_vect.fit_transform(unique_data["Text"].values)

In [243]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [244]:
final_counts.shape

(4986, 13510)

In [245]:
unique_data.shape

(4986, 10)

In [246]:
unique_data['Score'].value_counts()

1    4178
0     808
Name: Score, dtype: int64

In [247]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [248]:
!python -m pip install -U gensim

Requirement already up-to-date: gensim in /home/abhishek/anaconda3/lib/python3.7/site-packages (3.8.1)


In [249]:
random_sentence = unique_data['Text'].values[4985]
print(len(random_sentence))
from bs4 import BeautifulSoup

284


In [250]:
stop= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

## Preprocessing of "Text"

In [251]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

def remove_punc(my_str):
    no_punct = ""
    for char in my_str:
       if char not in punctuations:
           no_punct = no_punct + char
    return no_punct

from nltk.stem.snowball import SnowballStemmer

sno=SnowballStemmer('english')
# sno.stem("running")

In [252]:
i=0
output_=[]
all_positive_words=[]
all_negative_words=[]

for sent in unique_data['Text'].values:
    filtered_sentence=[]
    sent=remove_tags(sent)
    for w in sent.split():
        for cleaned_words in remove_punc(w).split():
            if ((cleaned_words.isalpha()) & (len(cleaned_words)>2)) :
                if (cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower()))
                    filtered_sentence.append(s)
                    if(unique_data['Score'].values)[i]==1:
                        all_positive_words.append(s)
                    if(unique_data['Score'].values[i]==0):
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
    str1=" ".join(filtered_sentence)
    output_.append(str1)
    i+=1


In [253]:
soup = BeautifulSoup(unique_data['Text'].values[0],'lxml')
text = soup.get_text()
print(text)

Why is this $[...] when the same product is available for $[...] here?http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDYThe Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [254]:
sent_0 = re.sub(r"http\S+", "", unique_data['Text'].values[0])
sent_0

'Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.'

In [255]:
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(unique_data['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stop)
    preprocessed_reviews.append(sentance.strip())

100%|██████████| 4986/4986 [00:01<00:00, 3155.41it/s]


In [256]:
preprocessed_reviews[1500]

'wow far two two star reviews one obviously no idea ordering wants crispy cookies hey sorry reviews nobody good beyond reminding us look ordering chocolate oatmeal cookies not like combination not order type cookie find combo quite nice really oatmeal sort calms rich chocolate flavor gives cookie sort coconut type consistency let also remember tastes differ given opinion soft chewy cookies advertised not crispy cookies blurb would say crispy rather chewy happen like raw cookie dough however not see taste like raw cookie dough soft however confusion yes stick together soft cookies tend not individually wrapped would add cost oh yeah chocolate chip cookies tend somewhat sweet want something hard crisp suggest nabiso ginger snaps want cookie soft chewy tastes like combination chocolate oatmeal give try place second order'

In [257]:
type((all_positive_words))

list

In [258]:
all_positive_words[0:10]

['product',
 'avail',
 'victor',
 'trap',
 'unreal',
 'cours',
 'total',
 'fli',
 'genocid',
 'pretti']

## Preprocessing of Summary

In [259]:
unique_data.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,1,1282953600,thirty bucks?,Why is this $[...] when the same product is av...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,1,1281052800,Flies Begone,We have used the Victor fly bait for 3 seasons...
1145,1244,B00002Z754,A3B8RCEI0FXFI6,B G Chase,10,10,1,962236800,WOW Make your own 'slickers' !,I just received my shipment and could hardly w...


In [260]:
type(output_)

list

In [261]:
output_[0]

'product avail victor trap unreal cours total fli genocid pretti stinki right nearbi'

In [262]:
unique_data['Text']=output_

In [263]:
unique_data.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,1,1282953600,thirty bucks?,product avail victor trap unreal cours total f...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,1,1281052800,Flies Begone,use victor fli bait season cant beat great pro...
1145,1244,B00002Z754,A3B8RCEI0FXFI6,B G Chase,10,10,1,962236800,WOW Make your own 'slickers' !,receiv shipment could hard wait tri product lo...


In [264]:
from tqdm import tqdm
preprocessed_summary = []
# tqdm is for printing the status bar
for sentance in tqdm(unique_data['Summary'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stop)
    preprocessed_summary.append(sentance.strip())

100%|██████████| 4986/4986 [00:01<00:00, 4909.93it/s]


In [265]:
i=0
output_summary=[]
unique_data['Summary']=preprocessed_summary
for sent in unique_data['Summary'].values:
    filtered_sentence=[]
    sent=remove_tags(sent)
    for w in sent.split():
        for cleaned_words in remove_punc(w).split():
            if ((cleaned_words.isalpha()) & (len(cleaned_words)>2)) :
                if (cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower()))
                    filtered_sentence.append(s)
                    if(unique_data['Score'].values)[i]==1:
                        all_positive_words.append(s)
                    if(unique_data['Score'].values[i]==0):
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
    str1=' '.join(filtered_sentence)
    output_summary.append(str1)
    i+=1


In [266]:
unique_data['Summary']=output_summary

In [267]:
print(unique_data.head(3)['Summary'])

2546        thirti buck
2547          fli begon
1145    wow make islick
Name: Summary, dtype: object


In [268]:
count_vect = CountVectorizer(ngram_range=(1,2))
output_bigram_counts = count_vect.fit_transform(unique_data['Text'].values)

In [269]:
output_bigram_counts.shape

(4986, 130767)

In [270]:
count_vect = CountVectorizer(ngram_range=(1,1))
output_bigram_counts = count_vect.fit_transform(unique_data['Text'].values)

In [271]:
output_bigram_counts.shape

(4986, 12522)

In [272]:
features=count_vect.get_feature_names()

In [273]:
len(features)

12522

In [274]:
output_bigram_counts[3,:].toarray()[0]

array([0, 0, 0, ..., 0, 0, 0])

## TF-IDF W2V

In [275]:
preprocessed_review = unique_data['Text']
model = TfidfVectorizer()
model1=model.fit_transform(preprocessed_review)
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [276]:
dictionary['aahhh']

8.821442624986176

In [277]:
len(model1[1,:].toarray()[0])

12522

In [283]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

is_your_ram_gt_16g=False
want_to_use_google_w2v = False
want_to_train_w2v = True

list_of_sentance=[]
for sentance in preprocessed_review:
    list_of_sentance.append(sentance.split())

w2v_model=Word2Vec(list_of_sentance,size=50,window=5,min_count=1,workers=4)
#print(w2v_model.wv.most_similar('great'))
#print('='*50)
w2v_words = list(w2v_model.wv.vocab)

In [285]:
tfidf_feat = model.get_feature_names()



tfidf_sent_vectors = []
row=0
for sent in tqdm(list_of_sentance):
    sent_vec = np.zeros(50)
    weight_sum = 0
    for word in sent:
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum!=0:
        sent_vec/=weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row+=1    

100%|██████████| 4986/4986 [00:27<00:00, 178.46it/s]


In [286]:
list_of_sentance1=[]
for sentance in preprocessed_review:
    list_of_sentance1.append(sentance.split())

In [287]:
list_of_sentance1[0]

['product',
 'avail',
 'victor',
 'trap',
 'unreal',
 'cours',
 'total',
 'fli',
 'genocid',
 'pretti',
 'stinki',
 'right',
 'nearbi']

array([ 0.11309557, -1.1946661 , -0.23537679,  1.3728076 , -2.2298326 ,
       -0.06668735,  0.14531259,  0.32801905,  0.6750246 , -0.43505388,
        1.0228213 ,  0.14003591,  0.836497  ,  0.08684575, -0.49848723,
        0.6034359 , -0.8757756 ,  0.7256371 ,  0.57720894, -0.49531662,
        1.0761774 ,  0.14654511,  0.77262086,  0.5278823 ,  0.1627704 ,
        0.29254645,  0.5472893 , -0.09734806,  0.10164458,  0.7542421 ,
       -0.69689363, -0.05714795, -1.830342  , -0.13195212,  0.7513145 ,
        0.8071091 ,  0.2986439 ,  0.11180231, -2.335677  ,  0.55025935,
        1.0886114 ,  1.5850269 ,  0.36554256,  0.3058795 ,  0.11210328,
       -0.32096022,  0.54893166, -0.57300043, -0.65999335,  1.0831542 ],
      dtype=float32)